# Kaggle - Titanic

Kaggle 제출 결과: ~76%

## import

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

## 학습데이터 호출

In [2]:
file = pd.read_csv("./data/titanic/train.csv")
file.shape

(891, 12)

## 데이터 전처리
1. 범위 지정
    - 데이터 범위: PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked
    - PassengerId, Name, Ticket값들은 수치값들이 아니니 필요없다.
    - Embarked는 약간 애매하다. 어느 장소로 부터 선박승선을 했냐는.. 의미있을까?
    - 상관계수:
        - 기본적으로 Fare, Sex, Pclass, Age는 상관계수가 높은편이다.
        - SibSp (Sibling, Spouse)와 Parch (Parents and children)은 애매하다.
            - 어느정도의 상관계수가 있되 높은편이 아니다. 가족이 survival rate를 높여준다는 추론은 가능?
        - Cabin은 어차피 Pclass1에 분류하는 사람들 (일부)만 갖고있다. 어떻게 보면 outlier.
            - Cabin있는 사람의 survival은 높긴 높다.
        - 현재 알아봐야 되는 사항은 상관계수와 Accuracy와 linear한 관계를 갖고있냐다.
            - 상관계수가 높은 자료를 사용하면 그만큼 Accuracy가 높아질까?
            - 상관계수가 높은 자료가 연속성자료이여서 범주형으로 바꾼다고 할떄:
                - 각 범주형 데이터의 상관계수가 일정하지 못한다면 학습에 지장을 줄까?
                - 연속성자료 -> 범주형 데이터 -> 상관계수가 높은 일부만 사용해도 될까?

In [3]:
train = pd.DataFrame(file, columns=["Survived", "Sex"])
train.head()

,Survived,Sex
0,0,male
1,1,female
2,1,female
3,1,female
4,0,male


In [4]:
train.isnull().sum()
# (train["Fare"]>0) & (train["Fare"]<10)

Survived    0
Sex         0
dtype: int64

In [5]:
Sex_dummies = pd.get_dummies(train['Sex'], prefix="Sex")
train = train.join(Sex_dummies)
train.drop('Sex', axis=1, inplace=True)
train.drop('Sex_male', axis=1, inplace=True)

In [6]:
train

,Survived,Sex_female
0,0,0
1,1,1
2,1,1
3,1,1
4,0,0
5,0,0
6,0,0
7,0,0
8,1,1
9,1,1


## 여기서 부턴 머신러닝!

In [7]:
dataset = train[:668].copy() # 75%
_y = dataset["Survived"].copy()
dataset.drop("Survived", axis=1, inplace=True)
x_data = MinMaxScaler().fit_transform(dataset.values)
y_data = _y.values.reshape(-1,1)

display(dataset.shape,
x_data.shape,
y_data.shape)

####
testset = train[668:].copy() # 75%
_y = testset["Survived"].copy()
testset.drop("Survived", axis=1, inplace=True)
x_test = MinMaxScaler().fit_transform(testset.values)
y_test = _y.values.reshape(-1,1)

display(testset.shape,
x_test.shape,
y_test.shape)

C:\Anaconda\envs\gpu_env\lib\site-packages\sklearn\utils\validation.py:590: DataConversionWarning: Data with input dtype uint8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


(668, 1)

(668, 1)

(668, 1)

C:\Anaconda\envs\gpu_env\lib\site-packages\sklearn\utils\validation.py:590: DataConversionWarning: Data with input dtype uint8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


(223, 1)

(223, 1)

(223, 1)

In [8]:
dataset.head()

,Sex_female
0,0
1,1
2,1
3,1
4,0


In [9]:
tf.reset_default_graph()

X = tf.placeholder(shape=[None,1], dtype=tf.float32)
Y = tf.placeholder(shape=[None,1], dtype=tf.float32)

W = tf.Variable(tf.random_normal([1,1]), name="weight")
b = tf.Variable(tf.random_normal([1]), name="bias")

logit = tf.matmul(X,W) + b
H = tf.sigmoid(logit)

cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=Y))
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(10000):
    _, c_val = sess.run([train, cost], feed_dict={X:x_data,Y:y_data})
    if step%1000==0:
        print("Cost: {}".format(c_val))
        
predict = tf.cast( H > 0.5, dtype=tf.float32 )
correct = tf.equal( predict, Y )
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

"""
Test data 생성
"""

percentage_1 = sess.run(accuracy, feed_dict={X:x_data, Y:y_data})
percentage_2 = sess.run(accuracy, feed_dict={X:x_test, Y:y_test})
print( "학습데이터에 대한 정확도: {}".format(percentage_1) )
print( "테스트데이터에 대한 정확도: {}".format(percentage_2) )
# while percentage < 0.85:
#     for step in range(10000):
#         _, c_val = sess.run([train, cost], feed_dict={X:x_data,Y:y_data})
#         if step%1000==0:
#             print("Cost: {}".format(c_val))
#     predict = tf.cast( H > 0.5, dtype=tf.float32 )
#     correct = tf.equal( predict, Y )
#     accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))
#     percentage = sess.run(accuracy, feed_dict={X:x_test, Y:y_test})

Cost: 0.896679162979126
Cost: 0.5321671962738037
Cost: 0.5222156643867493
Cost: 0.5189446210861206
Cost: 0.5173971652984619
Cost: 0.5166433453559875
Cost: 0.5162708759307861
Cost: 0.5160845518112183
Cost: 0.5159905552864075
Cost: 0.5159432291984558
학습데이터에 대한 정확도: 0.7859281301498413
테스트데이터에 대한 정확도: 0.7892376780509949


## TEST 준비

In [10]:
test_f = pd.read_csv("./data/titanic/test.csv")
test_f.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [11]:
test = pd.DataFrame(test_f, columns=["PassengerId", "Sex"])
test.head()

,PassengerId,Sex
0,892,male
1,893,female
2,894,male
3,895,male
4,896,female


In [12]:
_dummies = pd.get_dummies(test['Sex'], prefix="Sex")
test = test.join(_dummies)
test.drop('Sex', axis=1, inplace=True)
test.drop('Sex_male', axis=1, inplace=True)
test.head()

,PassengerId,Sex_female
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [13]:
enter = []

for p in test["PassengerId"]:
    result = sess.run(predict, feed_dict={X:test.loc[test["PassengerId"]==p,"Sex_female":]})
    if result[0]==0:
        enter.append([p,0])
    elif result[0]==1:
        enter.append([p,1])
    else:
        print(p, "um?")
        
submission = pd.DataFrame(enter, columns=["PassengerId","Survived"])

In [14]:
submission

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


In [38]:
submission.to_csv("./data/prediction_sex_only.csv", sep=',', index=False)